In [3]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
data = pd.read_csv('data/all_haiku.csv')

data['haiku'] = data['0']+' '+data['1']+' '+data['2']

data['haiku'] = data['haiku'].apply(lambda x: str(x).strip().replace('-',''))


In [17]:
#Leer los datos

text = ''

for i in data['haiku']:
    text += i + os.linesep

vocab = sorted(set(text))
    
print(f'Text is {len(text)} long')
print(f'Text has {len(vocab)} unique characters')

Text is 9860268 long
Text has 105 unique characters


In [21]:
#Procesamiento del texto

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [27]:
seq_length = 100
examples_per_epoch = len(text)//(seq_lenght+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

f
i
s
h
i


In [28]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))


'fishing boats colors of the rainbow\nash wednesday trying to remember  my dream\nsnowy morn pouring ano'
'ther cup of black coffee\nshortest day flames dance in the oven\nhaze half the horse hidden behind the '
'house\nlow sun the lady in red on high heels\nadvent the passing stranger farts\ntarn a bubble in the ic'
"e\nsnowflakes new asphalt in the holes\nCrystal Night'    gusts of rain       outside\nrain the sound of"
' a horse galloping through leaves\nwinter stars suddenly a whiff of perfume\nhungry half of the moon hi'


In [30]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))
    

Input data:  'fishing boats colors of the rainbow\nash wednesday trying to remember  my dream\nsnowy morn pouring an'
Target data:  'ishing boats colors of the rainbow\nash wednesday trying to remember  my dream\nsnowy morn pouring ano'


In [33]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print(f'Step     {i}')
    print(f'   input: {int(input_idx), repr(idx2char[input_idx])}')
    print(f'   expected output: {int(target_idx), repr(idx2char[target_idx])}')

Step     0
   input: (66, "'f'")
   expected output: (69, "'i'")
Step     1
   input: (69, "'i'")
   expected output: (79, "'s'")
Step     2
   input: (79, "'s'")
   expected output: (68, "'h'")
Step     3
   input: (68, "'h'")
   expected output: (69, "'i'")
Step     4
   input: (69, "'i'")
   expected output: (74, "'n'")
